In [1]:
import json

import boto3
from botocore.exceptions import ClientError

In [2]:
sts_client = boto3.client("sts")
account_id = sts_client.get_caller_identity()["Account"]
account_id

'268400237357'

In [3]:
def create_bucket(bucket_name, region=None):
    """Create an S3 bucket in a specified region

    If a region is not specified, the bucket is created in the S3 default
    region (us-east-1).

    :param bucket_name: Bucket to create
    :param region: String region to create bucket in, e.g., 'us-west-2'
    :return: True if bucket created, else False
    """

    # Create bucket
    try:
        if region is None:
            s3_client = boto3.client("s3")
            s3_client.create_bucket(Bucket=bucket_name)
        else:
            s3_client = boto3.client("s3", region_name=region)
            location = {"LocationConstraint": region}
            s3_client.create_bucket(
                Bucket=bucket_name, CreateBucketConfiguration=location
            )
    except ClientError as e:
        print(e)
        return False
    return True

In [4]:
s3 = boto3.resource("s3")
bucket_name = "spent-personalized"
s3_prefix = bucket_name + "/exports"
s3_path = "s3://" + s3_prefix

try:
    create_bucket(bucket_name="spent-personalized", region="ap-southeast-1")
except Exception as e:
    print(e)

An error occurred (BucketAlreadyOwnedByYou) when calling the CreateBucket operation: Your previous request to create the named bucket succeeded and you already own it.


In [5]:
iam_client = boto3.client("iam")

role_name = "createPinpointExportJob"
policy_name = role_name + "_policy"
policy_arn = ""


pinpoint_export_job_policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowUserToSeeBucketListInTheConsole",
            "Action": ["s3:ListAllMyBuckets", "s3:GetBucketLocation"],
            "Effect": "Allow",
            "Resource": ["arn:aws:s3:::*"],
        },
        {
            "Sid": "AllowRootAndHomeListingOfBucket",
            "Action": ["s3:ListBucket"],
            "Effect": "Allow",
            "Resource": [f"arn:aws:s3:::{bucket_name}"],
            "Condition": {
                "StringEquals": {"s3:delimiter": ["/"], "s3:prefix": ["", s3_prefix]}
            },
        },
        {
            "Sid": "AllowListingOfUserFolder",
            "Action": ["s3:ListBucket"],
            "Effect": "Allow",
            "Resource": [f"arn:aws:s3:::{bucket_name}"],
            "Condition": {"StringLike": {"s3:prefix": [s3_prefix]}},
        },
        {
            "Sid": "AllowAllS3ActionsInUserFolder",
            "Action": ["s3:*"],
            "Effect": "Allow",
            "Resource": [f"arn:aws:s3:::{s3_prefix}/*"],
        },
    ],
}

pinpoint_assume_role = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "pinpoint.amazonaws.com",
            },
            "Action": "sts:AssumeRole",
        }
    ],
}

# create role
try:
    create_role_res = iam_client.create_role(
        RoleName=role_name,
        AssumeRolePolicyDocument=json.dumps(pinpoint_assume_role),
        Description="This is a pinpoint role",
        Tags=[{"Key": "Owner", "Value": "spent-pinpoint"}],
    )
    print('create role success')
except ClientError as error:
    print(error)

# create policy
try:
    policy_res = iam_client.create_policy(
        PolicyName=policy_name, PolicyDocument=json.dumps(pinpoint_export_job_policy)
    )
    policy_arn = policy_res["Policy"]["Arn"]
    print('create policy success')
except ClientError as error:
    if error.response["Error"]["Code"] == "EntityAlreadyExists":
        print("Policy already exists")
        policy_arn = "arn:aws:iam::" + account_id + ":policy/" + policy_name
else:
    print("Role could not be created...")

# attch role policy
try:
    policy_attach_res = iam_client.attach_role_policy(
        RoleName=role_name, PolicyArn=policy_arn
    )
    print('attach policy success')
except ClientError as error:
    print(error)

print("create success")

An error occurred (EntityAlreadyExists) when calling the CreateRole operation: Role with name createPinpointExportJob already exists.
Policy already exists
attach policy success
create success


In [6]:
# list pinpoint application
pinpoint_client = boto3.client("pinpoint")

response = pinpoint_client.get_apps()

for item in response["ApplicationsResponse"]["Item"]:
    print(item["Name"], item["Id"])

spentmobileapp-dev d5ab8b3e1788464cbc67948ab042fbcc


In [7]:
response = iam_client.get_role(RoleName=role_name)

role_arn = response["Role"]["Arn"]
s3_path = "s3://" + s3_prefix
pinpoint_application_id = "d5ab8b3e1788464cbc67948ab042fbcc"

In [8]:
role_arn

'arn:aws:iam::268400237357:role/createPinpointExportJob'

In [15]:
response = pinpoint_client.create_export_job(
    ApplicationId=pinpoint_application_id,
    ExportJobRequest={
        "RoleArn": role_arn,
        "S3UrlPrefix": s3_path,
    },
)
response

{'ResponseMetadata': {'RequestId': 'd5c1094e-b750-4187-90a1-72aa7baed8af',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'content-type': 'application/json',
   'content-length': '251',
   'connection': 'keep-alive',
   'date': 'Sat, 23 Jan 2021 08:07:05 GMT',
   'x-amzn-requestid': 'd5c1094e-b750-4187-90a1-72aa7baed8af',
   'access-control-allow-origin': '*',
   'x-amz-apigw-id': 'Zl7eUF9VSQ0FgjQ=',
   'cache-control': 'no-store',
   'x-amzn-trace-id': 'Root=1-600bd928-68def719343a31a95b97a51a;Sampled=0',
   'x-cache': 'Miss from cloudfront',
   'via': '1.1 558358695c9b007bde310af96b06b375.cloudfront.net (CloudFront)',
   'x-amz-cf-pop': 'BKK50-C2',
   'x-amz-cf-id': 'SqqiVie0TSsk-7RQFOPRApdx3Y9bIhbdJehFApUnHZ52_GLJ8zqU3w=='},
  'RetryAttempts': 0},
 'ExportJobResponse': {'CreationDate': '2021-01-23T08:07:05.661Z',
  'Definition': {'RoleArn': 'arn:aws:iam::268400237357:role/createPinpointExportJob',
   'S3UrlPrefix': 's3://spent-personalized/exports'},
  'Id': 'be93c9cae8064441a1fef2e0013

In [16]:
export_job_id = response['ExportJobResponse']['Id']

status = ''
while(status != 'COMPLETED'):
    response = pinpoint_client.get_export_job(
        ApplicationId=pinpoint_application_id,
        JobId=export_job_id
    )
    status = response['ExportJobResponse']['JobStatus']
print('export completed', response)

export completed {'ResponseMetadata': {'RequestId': 'c6c56ed5-9b31-4903-97e2-8afd388fe138', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'application/json', 'content-length': '439', 'connection': 'keep-alive', 'date': 'Sat, 23 Jan 2021 08:07:12 GMT', 'x-amzn-requestid': 'c6c56ed5-9b31-4903-97e2-8afd388fe138', 'access-control-allow-origin': '*', 'x-amz-apigw-id': 'Zl7fnFZNyQ0FjyA=', 'cache-control': 'no-store', 'x-amzn-trace-id': 'Root=1-600bd930-5edfd69338995a587c6c33f4;Sampled=0', 'x-cache': 'Miss from cloudfront', 'via': '1.1 558358695c9b007bde310af96b06b375.cloudfront.net (CloudFront)', 'x-amz-cf-pop': 'BKK50-C2', 'x-amz-cf-id': 'UhgwBR3R8leHv7t32Vsi4ssLu01WhJ1W_AMml5kvmyO2jpk2SJJLcQ=='}, 'RetryAttempts': 0}, 'ExportJobResponse': {'ApplicationId': 'd5ab8b3e1788464cbc67948ab042fbcc', 'CompletedPieces': 1, 'CompletionDate': '2021-01-23T08:07:09.100Z', 'CreationDate': '2021-01-23T08:07:05.661Z', 'Definition': {'RoleArn': 'arn:aws:iam::268400237357:role/createPinpointExportJob